In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import json

os.listdir("../input")

# assignment of tweet to test or train
data_identification = pd.read_csv('/kaggle/input/dm19-lab2-nthu/data_identification.csv')

# emotion labels per tweet id
data_emotion = pd.read_csv('/kaggle/input/dm19-lab2-nthu/emotion.csv')

# raw data from twitter
data_tweets = pd.read_json('/kaggle/input/dm19-lab2-nthu/tweets_DM.json',lines=True)

# sample for competition upload
data_sampleSubmissions = pd.read_csv('/kaggle/input/dm19-lab2-nthu/sampleSubmission.csv')

data_sampleSubmissions.head()

In [ ]:
tweet = data_tweets["_source"]

import re
import string
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

#lemmatizer = WordNetLemmatizer() 
stemmer= PorterStemmer()

filtered_sentence2 = []

filtered_tweets = []
for text,j in zip(tweet,range(len(tweet))):
    text = tweet[j]['tweet']['text']
    
    # remove white spaces
    text = text.strip()
    
    
    # put hashtags at the beginning
    if tweet[j]['tweet']['hashtags']!=[]:
        text=' '.join(tweet[j]['tweet']['hashtags'])+" "+text
    # end inserted
    # to lower case
    text = text.lower()

    # remove numbers
    # it is considered that they are useless for the classificationt task
    text = re.sub(r'\d+', '', text)

    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # remove stop words
    word_tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))

    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    filtered_sentence1 = []
 
    for w in filtered_sentence: 
        if w not in stop_words: 
            filtered_sentence1.append(w)
   
    # do stemming
    i=0
    for word in filtered_sentence:
        #filtered_sentence1[i]=lemmatizer.lemmatize(word)
        filtered_sentence1[i]=stemmer.stem(word)
        i=i+1
        
    text =' '.join(filtered_sentence1)
    filtered_sentence2.append(text)

print(filtered_sentence2[0:5])

In [ ]:
# extract information from dictonary
tweet_id = [tweet[i]['tweet']['tweet_id'] for i in range(len(tweet))]
tweet_text = [tweet[i]['tweet']['text'] for i in range(len(tweet))]
tweet_hashtags = [tweet[i]['tweet']['hashtags'] for i in range(len(tweet))]

# create a dataframe with the important information and the preprocessed text
# keep original text, too
data = {'tweet_id':tweet_id, 'text':tweet_text, 'hashtags':tweet_hashtags}
df = pd.DataFrame(data)

df = pd.merge(df, data_identification, on='tweet_id',how='outer')
df = pd.merge(df, data_emotion, on='tweet_id',how='outer')

df["text_prep"]=filtered_sentence2
df.head()

In [ ]:
df.head(5)

In [ ]:
# save data as kaggle-output
df.to_pickle("data_porter.pkl")

In [ ]:
# create a link with the output data
from IPython.display import FileLink
FileLink(r'data_porter.pkl')